In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from functools import reduce

spark = SparkSession.builder.getOrCreate()

catalog = "workspace"
schema = "projeto_datatran"

anos = range(2007, 2025)

dfs = []

# Carregando todas as tabelas datatran para cada ano
for ano in anos:
    tabela = f"{catalog}.{schema}.datatran_{ano}"
    print(f"Carregando tabela RAW: {tabela}")

    df = spark.table(tabela)

# Convertendo todas as colunas para string
    for c in df.columns:
        df = df.withColumn(c, col(c).cast("string"))

    dfs.append(df)

print("Total de tabelas carregadas:", len(dfs))


In [0]:
# Unificando os dados
df_bronze = reduce(
    lambda a, b: a.unionByName(b, allowMissingColumns=True), 
    dfs
)

In [0]:
datatran_bronze = f"{catalog}.{schema}.datatran_unificada"

df_bronze.write.format("delta").mode("overwrite").saveAsTable(datatran_bronze)

print(f"Tabela unificada criada: {datatran_bronze}")